# # Sistema de clasificación de imágenes
El conjunto de datos se compone de fotos de perros y gatos proporcionadas como un subconjunto de fotos de uno mucho más grande de 3 millones de fotos anotadas manualmente. Estos datos se obtuvieron a través de una colaboración entre Petfinder.com y Microsoft.

El conjunto de datos se usó originalmente como un CAPTCHA, es decir, una tarea que se cree que un humano encuentra trivial, pero que una máquina no puede resolver, que se usa en sitios web para distinguir entre usuarios humanos y bots. La tarea se denominó "Asirra". Cuando se presentó "Asirra", se mencionó "que los estudios de usuarios indican que los humanos pueden resolverlo el 99,6% de las veces en menos de 30 segundos". A menos que se produzca un gran avance en la visión artificial, esperamos que los ordenadores no tengan más de 1/54.000 posibilidades de resolverlo.

En el momento en que se publicó la competencia, el resultado de última generación se logró con un SVM y se describió en un artículo de 2007 con el título "Ataques de Machine Learning contra el CAPTCHA de Asirra" (PDF) que logró una precisión de clasificación del 80%. Fue este documento el que demostró que la tarea ya no era una tarea adecuada para un CAPTCHA poco después de que se propusiera la tarea

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

In [ ]:
import os

dataset_path = "/home/vscode/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1"

# Listar archivos en la carpeta
files = os.listdir(dataset_path)
print(files)

In [ ]:
import os
import pandas as pd

# Ruta a la carpeta "train" que contiene "cats" y "dogs"
image_folder = os.path.join(dataset_path, "train")

# Recorrer subcarpetas y recolectar imágenes
image_paths = []
labels = []

for label in os.listdir(image_folder):
    class_folder = os.path.join(image_folder, label)
    if os.path.isdir(class_folder):
        for file in os.listdir(class_folder):
            if file.lower().endswith(".jpg"):
                image_paths.append(os.path.join(class_folder, file))
                labels.append(label)  # 'cats' o 'dogs'

# Crear el DataFrame
df = pd.DataFrame({
    "filepath": image_paths,
    "filename": [os.path.basename(path) for path in image_paths],
    "label": labels
})

# Verificar
print(df.shape)
df.head()

In [ ]:
# Guardar el DataFrame como CSV
import os
ruta="/workspaces/Clasficador-Imagenes/data"
train_csv_path = os.path.join(ruta, "train.csv")
df.to_csv(train_csv_path, index=False)

print(f"Archivo CSV guardado en: {train_csv_path}")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Obtener rutas de perros y gatos
dog_images = df[df["label"] == "dogs"]["filepath"].tolist()[:9]
cat_images = df[df["label"] == "cats"]["filepath"].tolist()[:9]

# Función para mostrar 9 imágenes
def show_images(image_paths, title):
    plt.figure(figsize=(10, 10))
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.axis("off")
    plt.suptitle(title)
    plt.show()

# Mostrar
show_images(dog_images, "Primeras 9 imágenes de perros")
show_images(cat_images, "Primeras 9 imágenes de gatos")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parámetros
image_size = (200, 200)
batch_size = 32

# Crear generador de datos con validación incluida
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Usamos el 20% para validación
)

# Generador para entrenamiento
train_generator= datagen.flow_from_directory(
    directory=image_folder,         # carpeta que contiene "cats" y "dogs"
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset='training',
    shuffle=True
)

# Generador para validación
val_generator = datagen.flow_from_directory(
    directory=image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset='validation',
    shuffle=False
)


In [ ]:
# Move the Data Through the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

model = Sequential()
model.add(Conv2D(input_shape = (200,200,3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Flatten())
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 1, activation = "sigmoid"))

In [ ]:
from keras.optimizers import Adam

model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

In [ ]:
model.fit(train_generator, epochs=1, steps_per_epoch=100)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("../models/vgg16_1.h5", monitor = "val_accuracy", verbose = 1, save_best_only = True, save_weights_only = False, mode = "auto")
early = EarlyStopping(monitor = "val_accuracy", patience = 3, verbose = 1, mode = "auto")
hist = model.fit(train_generator, steps_per_epoch = 100, validation_data =val_generator, validation_steps = 10, epochs = 3, callbacks = [checkpoint, early])

#Guardar el modelo
from pickle import dump

dump(model, open("../models/model_clasificador_42.sav", "wb"))

In [ ]:
# Check the Accuracy of the Data
import matplotlib.pyplot as plt

# Plot the Results
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])

# Configure the Plot Layout
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "Loss", "Validation Loss"])

# Plot
plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Cargar el modelo guardado
model = load_model("../models/vgg16_1.h5")

# Ruta de una imagen para probar (puedes cambiar la ruta por otra imagen del set)
img_path = df.iloc[200]['filepath']  # o puedes poner la ruta manualmente como: "../data/raw/train/cat.123.jpg"

# Preprocesar la imagen
img = image.load_img(img_path, target_size=(200, 200))  # Asegúrate de usar el mismo tamaño que usaste para entrenar
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Hacer predicción
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)
class_labels = list(train_generator.class_indices.keys())

print("Clase predicha:", class_labels[predicted_class])
print("Probabilidades:", prediction)

# Mostrar la imagen
plt.imshow(img)
plt.title(f"Predicción: {class_labels[predicted_class]}")
plt.axis("off")
plt.show()
